# Traitement des données pour un fichier

Le but de ce fichier est de présenter le pré-traitement des données, avant l'utilisation d'un module d'IA.<br>
Pour réliser cela il faut :
- réduire au maximum le nombre de points pour permettre un apprentissage efficace même sur peu de données
- conserver toutes les informations permettant de différencier une courbe normale d'une courbe de malade


Import des modules utiles :
- `pandas` pour le traitement des données 
- `numpy` pour les calcules sur les vecteurs
- `scipy.signal` pour faire du traitement du signal

Remarque : on utilisera `plotly` pour l'affichage des courbes, il faut donc l'avoir installé.

In [57]:
import pandas as pd
import numpy as np
from importlib import reload
from inspect import getsource

import tools

pd.options.plotting.backend = "plotly"
reload(tools)

<module 'tools' from 'c:\\Users\\jleva\\Documents\\Telecom\\2A\\stage\\GCMS\\tools.py'>

Définition des constantes qui seront utiles dans la suite du programme :
- `PATH`, le chemin jusqu'au fichier csv étudié
- `PERIOD`, la periode de ré-échantillonage des données en milliseconde, permettant de conserver plus ou moins de points sur la courbe
- `INTERVAL`, l'intervalle de temps contenant les informations significatives en minutes
- `NOISE_FREQ`, fréquence de coupure du filtre passe-bas permettant d'éliminer le bruit en Hz, valeur de None si il n'y a pas de filtrage souhaité
- `BIAS_FREQ`, fréquence de coupure du filtre passe-bas permettant de déterminer le biais sur la courbe en Hz
- `THRESHOLD`, seuil à partir duquel on considère une valeur comme significative dans la mesure.
- `SPIKES`, temps d'apparition en minute des trois pics de référence, valeur de None si le pic n'apparait pas

In [58]:
PATH = '../donnees/export-chromato/AubinM-20201109-CHROMATOGRAM.CSV'
print(tools.PERIOD) 
print(tools.RESAMPLE_MS)
print(tools.INTERVAL)
print(tools.NOISE_FREQ)
print(tools.BIAS_FREQ) 
print(tools.THRESHOLD)
print(tools.SPIKES) 

4000
4000ms
[6, 45]
None
0.05
1.7
[21.5, 25.6, 30.2]


## Lecture d'un fichier csv et affichage des données
On utilise pandas pour lire directement le fichier csv et mettre les données dans un objet du type DataFrame.<br>
Les 5 premières lignes ne sont pas prises en compte car elles correspondent aux métadonnées de l'analyse. <br>
Le parser détecte trois colonnes car les lignes finissent par une virgule, il faut donc supprimer la troisième colonne qui est vide du DataFrame.

In [59]:
df = tools.readCSV(PATH)

On ne garde que les valeurs comprises dans l'intervalle qui nous interesse, cela supprime aussi les cases éventuellement vide du csv (temps de 0 à la lecture)<br>
Affichage des données brutes

Remarque : dans l'exemple on applique cela sur un DataFrame temporaire `temp` car cela est déjà fait dans la fonction `adaptCurve()`

In [60]:
temp = df.drop(df[df.index > tools.INTERVAL[1]].index)
temp = temp.drop(temp[temp.index < tools.INTERVAL[0]].index)
temp.plot()

## Mise en évidence des pics
Le passage des données en logarithme permet de mettre en évidence les plus petits pics du graphique, le but est de ne pas donner un importance trop grande aux pics les plus grands, car cela pourrait compliquer le travail sur la partie IA dans le cas ou l'information importante est contenue dans un pic de taille faible.<br>
Remarque : la fonction logarithme est bijective, il n'y a donc pas de perte d'information.

In [61]:
temp = np.log(temp)
temp.plot()

Ensuite on fait une normalisation des points de la courbe

In [62]:
temp = (temp - temp.mean())/temp.std()
temp.plot()

## Intervalles de mesures constant
La machine faisant les mesures n'a pas une période d'échantillonage constante, cela est problématique pour l'exploitation des données car chaque courbes aura des points d'échantillonage différents. <br>
On utilise donc la fonction `resample` de `pandas` qui permet de donner un échantilonnage constant Associé à `max` qui permet de conserver le maximum des points qui se trouveront entre deux points d'échantilonnage, dans le but de conserver la taille des différents pics.<br>
Ensuite `interpolate` permet de faire une interpolation polynomiale permettant de completer les points qui n'ont pas encore de valeur.

Avant de faire ceci il faut commencer par convertir le temps qui est un valeur en minutes dans un format compréhensible par `pandas`.

In [63]:
newdf = tools.adaptCurve(df)
print('nouveau nombre de valeur : ', len(newdf), '  ancien nombre de valeur : ', len(df))
newdf.plot()

nouveau nombre de valeur :  585   ancien nombre de valeur :  9149


## Filtrage et suppression du biais
On remarque sur la courbe ci-dessus que la machine à tendance à ajouter un biais au cours du temps. Les valeurs en 0 sont proche de -1 alors qu'en 45 elle sont plutot proche de 0.4. <br>
L'idée est donc de récupérer la composante basse fréquence de la courbe correspondant à ce biais par un filtre passe-bas, et ensuite de la soustraire aux valeurs de la courbe pour le comprenser. <br>
Une second filtre passe-bas pourra être utilisé pour supprimer les hautes fréquences, éliminant le bruit de mesure.

Mise en place du filtre passe-bas avec `butter` de `scipy`.<br>
Tiré de : <https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units>

In [64]:
newdf['bias'] = tools.butter_lowpass_filter(newdf, tools.BIAS_FREQ, 1/tools.PERIOD*1000)
print(getsource(tools.butter_lowpass_filter))
newdf.plot()

def butter_lowpass_filter(data : pd.DataFrame, cutoff : float, fs : float, order=5) -> pd.DataFrame:
    """Application du filtre."""
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y



Filtrage des hautes fréquences et suppression du biais :

In [65]:
if (tools.NOISE_FREQ is None):
    newdf['corrected'] = newdf['values'] - newdf['bias']
else :
    newdf['corrected'] = tools.butter_lowpass_filter(newdf['values'], tools.NOISE_FREQ, 1/tools.PERIOD*1000) # on perd la hauteur relative entre les pics parfois
    newdf['corrected'] = newdf['corrected'] - newdf['bias']
newdf.plot()

Pour éliminer les données parasites restantes, l'idée est de faire un seuillage pour ne conserver que les valeurs significatives sur la courbe et ainsi éliminer l'impact du bruit de mesure.

In [66]:
newdf.loc[newdf['corrected'] < tools.THRESHOLD, 'corrected'] = 0
print('Nombre de valeurs restantes : ', len(newdf['corrected']))
newdf['corrected'].plot()

Nombre de valeurs restantes :  585


Pour faire tous les traitements d'un coup, il y a la fonction `readAndAdaptDataFromCSV`, utilisée comme suit : 

In [68]:
other = tools.readAndAdaptDataFromCSV(PATH)
print(other)
other.plot()

                          values
time                            
0 days 00:06:00.160020  2.923787
0 days 00:06:04.160020  2.039405
0 days 00:06:08.160020  0.000000
0 days 00:06:12.160020  0.000000
0 days 00:06:16.160020  0.000000
...                          ...
0 days 00:44:40.160020  0.000000
0 days 00:44:44.160020  0.000000
0 days 00:44:48.160020  0.000000
0 days 00:44:52.160020  0.000000
0 days 00:44:56.160020  0.000000

[585 rows x 1 columns]
